<a href="https://colab.research.google.com/github/KevinAkat/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS 

# Uploading the file w/ files.upload() function

from google.colab import files
# uploaded = files.upload()

Saving house-votes-84.data to house-votes-84.data


In [0]:
# Importing pandas and numpy

import pandas as pd
import numpy as np

In [0]:
# Reading in the file into a dataframe called "df_votes"
# Inspecting the shape and head of df_votes

df_votes = pd.read_csv('house-votes-84.data')

print(df_votes.shape)
df_votes.head()



(434, 17)


,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
# Checking datatypes

df_votes.dtypes

republican    object
n             object
y             object
n.1           object
y.1           object
y.2           object
y.3           object
n.2           object
n.3           object
n.4           object
y.4           object
?             object
y.5           object
y.6           object
y.7           object
n.5           object
y.8           object
dtype: object

In [0]:
# Adding column headers and re-checking the head of the DF

column_headers = ['Party','handicapped-infants','water-project',
                  'adopting-budget-res','physician-fee-freeze',
                  'el-salvador-aid','religious-groups-in-schools',
                  'anti-satellite-test-ban','aid-to-nicaraguan-contras',
                  'mix-missile','immigration','synfuels-corporation cutback',
                  'education-spending','superfund-right-to-sue','crime',
                  'duty-free-exports','south-africa']

df_votes = pd.read_csv('house-votes-84.data', names=column_headers)

df_votes.head()

,Party,handicapped-infants,water-project,adopting-budget-res,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mix-missile,immigration,synfuels-corporation cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
# Using the replace function to conver 'n' into 0, 'y' into 1 and '?' into NaN
# values. This makes performing hypothesis testing possible.

df_votes = df_votes.replace({'n': 0,'y': 1,'?': np.NaN})

df_votes.head()

,Party,handicapped-infants,water-project,adopting-budget-res,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mix-missile,immigration,synfuels-corporation cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


###2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01###

In [0]:
# Creating the conditions to seperate out Republicans and Democrats
# into 2 different Dataframes. 

rep_condition = df_votes["Party"] == 'republican'
dem_condition = df_votes["Party"] == 'democrat'

# Seperating dems and reps into 2 differnet DFs

rep_df = df_votes[rep_condition]
dem_df = df_votes[dem_condition]


# Checking the heads
# rep_df.head()
dem_df.head()






,Party,handicapped-infants,water-project,adopting-budget-res,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mix-missile,immigration,synfuels-corporation cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


**Null:** Democrats do not support adopting the budget resolution more than republicans
(Dem x-bar == Rep x-bar)

**Alt:**  Democrats do not support adopting the budget resolution more than republicans (Dem x-bar != Rep x-bar)

**Confidence Interval** = 99%
**Alpha** = .01

In [0]:
# Importing stats from scipy

from scipy import stats

# Running a 2-sample T-Test on the Adopted Budget Resolution Bill 
# to see the level of Democrats and Republic bipartisanship, on the issue
# or lack thereof.

stats.ttest_ind(dem_df['adopting-budget-res'], rep_df['adopting-budget-res'], nan_policy='omit')


Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

P = 2.0703402795404463e-77
Alpha = .01

Since p <= alpha, I reject the null hypothesis. In other words, I **reject** the notion that *Demoncrats do not support the budget resolution more than Republicans*. We can say with 99% certainty that the means between Democratic and Republican voting on this bill are unequal. 

###3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01###

**Null:** Republicans do not support Superfund's right to sue more than Democrats
(Rep x-bar == Dem x-bar)

**Alt:**  Republicans do support adopting the budget resolution more than Democrats (Rep x-bar != Dem x-bar)

**Confidence Interval** = 99%
**Alpha** = .01

In [0]:
# Running a 2-sample T-Test on the Bill that would allow Superunds the right to sue.
# This helps determine the level of Democrats and Republic bipartisanship on the issue,
# or lack thereof.

stats.ttest_ind(rep_df['superfund-right-to-sue'], dem_df['superfund-right-to-sue'], nan_policy='omit')

Ttest_indResult(statistic=13.51064251060933, pvalue=1.2278581709672758e-34)

P = 1.2278581709672758e-34
Alpha = .01

Since p <= alpha, I reject the null hypothesis. In other words, I **reject** the notion that *Republicans do not a Superund's right to sue more than Democrats*. We can say with 99% certainty that the means between Democratic and Republican voting on this bill are unequal. 

###4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

**Null:** Republicans and Democrats equally agree on the Water Project bill. (Dem x-bar == Rep x-bar) 

**Alt:**  Republicans and Democrats do not equally agreed on the Water Project bill. (Dem x-bar != Rep x-bar)


**Confidence Interval** = 90%
**Alpha** = .10

In [0]:
# Running a 2-sample T-Test on the Water Project Bill.
# This helps determine the level of Democrats and Republic bipartisanship on the issue,
# or lack thereof.

stats.ttest_ind(rep_df['water-project'], dem_df['water-project'], nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

P = 0.9291556823993485
Alpha = .10

Since p > alpha, I failt to reject the null hypothesis. In other words, I **failt to reject** the notion that *Republicans and Democrats equally agree on The Water Bill*. We cannot say with 90% certainty that the means between Democratic and Republican voting on this bill are unequal. 